In [12]:
import os
from pathlib import Path

import cv2 as cv
import matplotlib
from dotenv import load_dotenv

from utils.edge import tooth_isolation
from utils.yolo import get_teeth_ROI

# from ultralytics import YOLO

load_dotenv()
matplotlib.use('module://matplotlib_inline.backend_inline')

% matplotlib inline

YOLO_model_dir = Path(os.getenv('YOLO_MODEL_DIR'))
data_dir = Path(os.getenv('DATASET_DIR')) / 'phase-3'


In [14]:
import torch
from glob import glob

model = torch.hub.load(YOLO_model_dir / '..', 'custom', path=YOLO_model_dir / '8-bound.pt', source='local')
# Image
images = glob(str(data_dir / '*.jpg'))

images


requirements: /opt/miniconda3/envs/pt113/lib/python3.9/site-packages/yolov5/requirements.txt not found, check failed.


['/Users/lucyxu/PycharmProjects/datasets/phase-3/00008276.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008317.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008329.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008328.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008314.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008270.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008338.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008339.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008271.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008298.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008306.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008272.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008413.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008228.jpg',
 '/Users/lucyxu/PycharmProjects/datasets/phase-3/00008200.jpg',
 '/Users/lucyxu/PycharmProjects/datasets

In [15]:
results = model(images[:])


In [16]:
save = False
rotation_fix = False
yolov8 = True


In [38]:
teeth_roi = get_teeth_ROI(results)

split_teeth = teeth_roi['split_teeth']
teeth_roi = teeth_roi['images']

tooth_position_dict = {
    0: 'left',
    1: 'middle',
    2: 'right'
}

teeth_region = {}
for file_name, data in teeth_roi.items():
    temp_region = {}
    for datum in data:
        flag = datum['flag']
        im = datum['image']
        offset = datum['offset']
        tooth_position = datum['tooth_position']

        if im.size == 0:
            continue

        im_g = cv.cvtColor(im, cv.COLOR_RGB2GRAY)

        isolation_data = tooth_isolation(im_g, flag=flag, filename=file_name, tooth_position=tooth_position,
                                         save=save,
                                         rotation_fix=rotation_fix)
        region = isolation_data['crop_regions']
        for k, v in region.items():
            xyxy = region[k]['xyxy']

            xyxy[0] += offset[0]
            xyxy[1] += offset[1]
            xyxy[2] += offset[0]
            xyxy[3] += offset[1]

            region[k]['xyxy'] = xyxy

        temp_region.update(region)
        temp_region.update(split_teeth[file_name])

        teeth_region[file_name] = temp_region

# Display bounding area
# for file_name, data in teeth_region.items():
#     im0 = cv.imread(f'../../Datasets/phase-3/{file_name}.jpg')
#     annotator = Annotator(im0, line_width=3, example=file_name)
#     for tooth_number, body in data.items():
#         xyxy = body['xyxy']
#         annotator.box_label(xyxy, str(tooth_number), color=(255, 0, 0))
#
#     im1 = annotator.result()
#
#     plt.imshow(im1)
#     plt.title(file_name)
#     plt.show()


In [71]:
for file_name, region in teeth_region.items():
    file_content = ''
    for tooth_number, data in region.items():
        xyxy = data['xyxy'].reshape([4]).int()
        x1, x2, y1, y2 = xyxy

        # print(xyxy.reshape([4]))

        file_line = f'{tooth_number} 0.8 {x1} {x2} {y1} {y2} \n'

        file_content += file_line

    with open(f'../evals/Ours/{file_name}.txt', 'w') as f:
        f.write(file_content)


In [73]:
a = set(teeth_roi.keys())
b = set(teeth_region.keys())

c = a - b

for i in c:
    with open(f'../evals/Ours/{i}.txt', 'w') as f:
        f.write('')
